In [1]:
from sklearn.cross_decomposition import PLSRegression
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [2]:
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/x_train.csv'
x_train = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/y_train.csv'
y_train = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/x_test.csv'
x_test = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/y_test.csv'
y_test = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Datasets/test.csv'
test = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/Submission%20Test.csv'
submission_test = pd.read_csv(url)

In [3]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.fit_transform(x_test)

In [6]:
param_grid = {'n_components': [5, 10], 'max_iter': [1000, 5000]}
grid = GridSearchCV(PLSRegression(), param_grid, refit=True, verbose=2)
grid.fit(x_train.to_numpy(), y_train.to_numpy().ravel())
print(grid.best_estimator_)
grid_predictions = grid.predict(x_test.to_numpy())

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ......................max_iter=1000, n_components=5; total time=   0.5s
[CV] END ......................max_iter=1000, n_components=5; total time=   0.8s
[CV] END ......................max_iter=1000, n_components=5; total time=   0.6s
[CV] END ......................max_iter=1000, n_components=5; total time=   0.6s
[CV] END ......................max_iter=1000, n_components=5; total time=   0.7s
[CV] END .....................max_iter=1000, n_components=10; total time=   1.3s
[CV] END .....................max_iter=1000, n_components=10; total time=   0.8s
[CV] END .....................max_iter=1000, n_components=10; total time=   0.8s
[CV] END .....................max_iter=1000, n_components=10; total time=   0.8s
[CV] END .....................max_iter=1000, n_components=10; total time=   0.9s
[CV] END ......................max_iter=5000, n_components=5; total time=   0.6s
[CV] END ......................max_iter=5000, n_c

In [7]:
Id = test['Store'].astype('string') + '_' + test['Dept'].astype('string') + '_' + test['Date'].astype('string')
Id

0           1_1_2012-11-02
1           1_1_2012-11-09
2           1_1_2012-11-16
3           1_1_2012-11-23
4           1_1_2012-11-30
                ...       
115059    45_98_2013-06-28
115060    45_98_2013-07-05
115061    45_98_2013-07-12
115062    45_98_2013-07-19
115063    45_98_2013-07-26
Length: 115064, dtype: string

In [8]:
y_pred = grid.predict(submission_test)
y_pred

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(


array([[23.21662902],
       [23.22568926],
       [23.23474951],
       ...,
       [15.57079529],
       [15.58891578],
       [15.59797603]])

In [13]:
weekly_sales = np.round(y_pred[:,0]**3, 2)
weekly_sales

array([12514.04, 12528.69, 12543.36, ...,  3775.13,  3788.33,  3794.94])

In [14]:
sub = {'Id':Id, 'Weekly_Sales':weekly_sales}
PLS_submission = pd.DataFrame(data=sub)
PLS_submission

,Id,Weekly_Sales
0,1_1_2012-11-02,12514.04
1,1_1_2012-11-09,12528.69
2,1_1_2012-11-16,12543.36
3,1_1_2012-11-23,12558.04
4,1_1_2012-11-30,12572.73
...,...,...
115059,45_98_2013-06-28,3761.97
115060,45_98_2013-07-05,3768.55
115061,45_98_2013-07-12,3775.13
115062,45_98_2013-07-19,3788.33


In [15]:
from google.colab import drive
drive.mount('drive')
PLS_submission.to_csv('PLS_submission.csv')
!cp PLS_submission.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
